# Markdown Brochure Generator

In this small exercise I use [Gradio](https://www.gradio.app/) in order to implement a simple Markdown brochure generator for an website.

In [2]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from google import genai
from google.genai import types

import gradio as gr

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCN


In [4]:
openai = OpenAI()
claude = anthropic.Anthropic(api_key=anthropic_api_key)
gemini = genai.Client(api_key=google_api_key)

In [5]:
SYSTEM_PROMPT = "You are an assistant that analyzes the contents of a website landing page and creates a short brochure about the company for prospective customers, investors and recruits. Include details of company culture, customers and careers/jobs if you have the information. Respond in Markdown."

In [6]:
def gpt_stream_answer(prompt, system_message):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]

    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [7]:
def claude_stream_answer(prompt, system_message):
    result = claude.messages.stream(
        model="claude-3-5-haiku-latest",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [8]:
def gemini_stream_answer(prompt, system_message):
    stream = gemini.models.generate_content_stream(
        model="gemini-2.5-flash",
        contents=prompt,
        config=types.GenerateContentConfig(system_instruction=system_message, temperature=0.7)
    )

    result = ""
    for chunk in stream:
        result += chunk.text or ""
        yield result

In [9]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [10]:
def customized_system_prompt(preference = "", system_initial = SYSTEM_PROMPT):
    if preference == "Professional":
        system_initial += "\nYour answer is very professional and corporate oriented. You answer is paragraphs and bulletpoints that describe clearly the company. You DO NOT use emojies, slangs or any other unprofessional language. You emphasise the history of the company and the product (or main thing) that they are representing. You report links and other references. Your tone is simple and professional, like the CEO of the company."
    elif preference == "Funny":
        system_initial += "\nYour answer is hunny and friendly at the same time. You use emojies to capture the attention and cute, friendly, short words that seems to explain things faster. You answer is small text and bulletpoints. Your answer is like you talking or promoting the company to a friend in a friendly and gentle way."
    elif preference == "Statistical":
        system_initial += "\nYour answer is mainly focust on statistical data. You role is to better represent data and numbers in the best way. You tone is simple, neutral and result-oriented. You do not focus on the history or employee, instead you use the company and its role to better explain and represent the company's data. You do not use emojies and you answer mostly in paragraphs."
    elif preference == "Sarcastic":
        system_initial += "\nYour answer is sarcastic and contains a lot of small jokes and double-meaning about the company. You do not want to offend them, but the brochure you are creating is roasting them a bit. You maintain the respect about the company and your only goal of the brochure is to talk about the company and make people smaile and laugh a bit. You can use silly and funny emojie around the texts and you answer is mixed between paragraphs, some lists and maybe punhlines."

    return system_initial


In [11]:
def stream_brochure(company_name, url, model, tone):
    system_message = customized_system_prompt(tone, SYSTEM_PROMPT)

    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model == "GPT":
        result = gpt_stream_answer(prompt, system_message)
    elif model == "Claude":
        result = claude_stream_answer(prompt, system_message)
    elif model == "Gemini":
        result = gemini_stream_answer(prompt, system_message)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name : "),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select Model", value="GPT"),
        gr.Dropdown(["Professional", "Funny", "Statistical", "Sarcastic"], label="Select the tone of the brochure :", value="Professional")
    ],
    outputs=[gr.Markdown(label="Brochure :")],
    flagging_mode="never"
)
view.launch(inbrowser=True)

In [14]:
view.close()

Closing server running on port: 7860
